# Zurich Real Estate Price Prediction - Exploratory Data Analysis

This notebook explores and analyzes the Zurich real estate datasets to better understand price patterns and generate insights for our machine learning model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## Load Data

Let's load the raw CSV files and examine their structure.

In [ ]:
# Load neighborhood dataset
neighborhood_path = os.path.join('..', 'data', 'raw', 'bau515od5155.csv')
neighborhood_df = pd.read_csv(neighborhood_path)

# Display dataset info
print(f"Neighborhood dataset shape: {neighborhood_df.shape}")
neighborhood_df.head()

In [ ]:
# Load building age dataset
building_age_path = os.path.join('..', 'data', 'raw', 'bau515od5156.csv')
building_age_df = pd.read_csv(building_age_path)

# Display dataset info
print(f"Building age dataset shape: {building_age_df.shape}")
building_age_df.head()

## Initial Data Processing

Let's rename the columns to make them more interpretable and filter for the data we need.

In [ ]:
# Define column mapping for neighborhood data
neighborhood_column_map = {
    'Stichtagdatjahr': 'year',
    'RaumLang': 'neighborhood',
    'AnzZimmerLevel2Lang_noDM': 'room_count',
    'HAMedianPreis': 'median_price',
    'HAPreisWohnflaeche': 'price_per_sqm',
    'HAArtLevel1Lang': 'property_type'
}

# Select needed columns and rename
n_df = neighborhood_df.rename(columns=neighborhood_column_map)[list(neighborhood_column_map.values())]

# Define column mapping for building age data
building_age_column_map = {
    'Stichtagdatjahr': 'year',
    'BaualterLang_noDM': 'building_age',
    'AnzZimmerLevel2Lang_noDM': 'room_count',
    'HAMedianPreis': 'median_price',
    'HAPreisWohnflaeche': 'price_per_sqm',
    'HAArtLevel1Lang': 'property_type'
}

# Select needed columns and rename
ba_df = building_age_df.rename(columns=building_age_column_map)[list(building_age_column_map.values())]

# Filter for apartments (Wohnungen) only
if 'property_type' in n_df.columns:
    n_df = n_df[n_df['property_type'] == 'Wohnungen']
    
if 'property_type' in ba_df.columns:
    ba_df = ba_df[ba_df['property_type'] == 'Wohnungen']

# Convert price columns to numeric
n_df['median_price'] = pd.to_numeric(n_df['median_price'], errors='coerce')
n_df['price_per_sqm'] = pd.to_numeric(n_df['price_per_sqm'], errors='coerce')

ba_df['median_price'] = pd.to_numeric(ba_df['median_price'], errors='coerce')
ba_df['price_per_sqm'] = pd.to_numeric(ba_df['price_per_sqm'], errors='coerce')

print(f"Processed neighborhood data shape: {n_df.shape}")
n_df.head()

In [ ]:
print(f"Processed building age data shape: {ba_df.shape}")
ba_df.head()

## Data Overview and Summary

Let's examine some basic statistics about our dataset.

In [ ]:
# Check unique values in key categorical columns
print(f"Years in data: {sorted(n_df['year'].unique())}")
print(f"Number of neighborhoods: {n_df['neighborhood'].nunique()}")
print(f"Room count categories: {sorted(n_df['room_count'].unique())}")
print(f"Building age categories: {sorted(ba_df['building_age'].unique())}")

## Price Distribution Analysis

In [ ]:
# Overall price distribution for the latest year
latest_year = n_df['year'].max()
latest_data = n_df[n_df['year'] == latest_year]

plt.figure(figsize=(10, 6))
sns.histplot(latest_data['median_price'].dropna(), kde=True, bins=30)
plt.title(f'Distribution of Median Property Prices in Zurich ({latest_year})')
plt.xlabel('Median Price (CHF)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Price by room count
plt.figure(figsize=(12, 6))
sns.boxplot(x='room_count', y='median_price', data=latest_data)
plt.title(f'Median Price by Room Count ({latest_year})')
plt.xlabel('Room Count')
plt.ylabel('Median Price (CHF)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Price by building age (from building age dataset)
latest_ba_year = ba_df['year'].max()
latest_ba_data = ba_df[ba_df['year'] == latest_ba_year]

plt.figure(figsize=(14, 6))
sns.boxplot(x='building_age', y='median_price', data=latest_ba_data)
plt.title(f'Median Price by Building Age ({latest_ba_year})')
plt.xlabel('Building Age')
plt.ylabel('Median Price (CHF)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Price Trends Over Time

In [ ]:
# Calculate average prices by year
yearly_prices = n_df.groupby('year')['median_price'].mean().reset_index()

plt.figure(figsize=(12, 6))
plt.plot(yearly_prices['year'], yearly_prices['median_price'], marker='o', linewidth=2)
plt.title('Average Property Price Trend in Zurich (2009-2024)')
plt.xlabel('Year')
plt.ylabel('Average Median Price (CHF)')
plt.grid(True)
plt.xticks(yearly_prices['year'])
plt.tight_layout()
plt.show()

In [ ]:
# Price trends by room count
room_count_trends = n_df.groupby(['year', 'room_count'])['median_price'].mean().reset_index()

plt.figure(figsize=(14, 8))
for room_count in sorted(room_count_trends['room_count'].unique()):
    room_data = room_count_trends[room_count_trends['room_count'] == room_count]
    plt.plot(room_data['year'], room_data['median_price'], marker='o', label=f'{room_count} rooms')

plt.title('Price Trends by Room Count (2009-2024)')
plt.xlabel('Year')
plt.ylabel('Average Median Price (CHF)')
plt.grid(True)
plt.legend()
plt.xticks(sorted(n_df['year'].unique()))
plt.tight_layout()
plt.show()

## Neighborhood Analysis

In [ ]:
# Top 10 most expensive neighborhoods
top_neighborhoods = latest_data.groupby('neighborhood')['median_price'].mean().sort_values(ascending=False).head(10)

plt.figure(figsize=(14, 8))
top_neighborhoods.plot(kind='bar')
plt.title(f'Top 10 Most Expensive Neighborhoods in Zurich ({latest_year})')
plt.xlabel('Neighborhood')
plt.ylabel('Average Median Price (CHF)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Price per square meter by neighborhood
if 'price_per_sqm' in latest_data.columns:
    sqm_price = latest_data.groupby('neighborhood')['price_per_sqm'].mean().sort_values(ascending=False).head(10)
    
    plt.figure(figsize=(14, 8))
    sqm_price.plot(kind='bar')
    plt.title(f'Top 10 Neighborhoods by Price per Square Meter ({latest_year})')
    plt.xlabel('Neighborhood')
    plt.ylabel('Average Price per Square Meter (CHF)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## Room Count Analysis

In [ ]:
# Price per square meter by room count
if 'price_per_sqm' in latest_data.columns:
    room_sqm_price = latest_data.groupby('room_count')['price_per_sqm'].mean().sort_values(ascending=False)
    
    plt.figure(figsize=(12, 6))
    room_sqm_price.plot(kind='bar')
    plt.title(f'Price per Square Meter by Room Count ({latest_year})')
    plt.xlabel('Room Count')
    plt.ylabel('Average Price per Square Meter (CHF)')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## Building Age Analysis

In [ ]:
# Price trends by building age
age_trends = ba_df.groupby(['year', 'building_age'])['median_price'].mean().reset_index()

plt.figure(figsize=(14, 8))
for age in sorted(age_trends['building_age'].unique()):
    age_data = age_trends[age_trends['building_age'] == age]
    plt.plot(age_data['year'], age_data['median_price'], marker='o', label=age)

plt.title('Price Trends by Building Age (2009-2024)')
plt.xlabel('Year')
plt.ylabel('Average Median Price (CHF)')
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(sorted(ba_df['year'].unique()))
plt.tight_layout()
plt.show()

## Correlation Analysis

In [ ]:
# Scatterplot of median price vs price per square meter
if 'price_per_sqm' in latest_data.columns:
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='price_per_sqm', y='median_price', hue='room_count', data=latest_data)
    plt.title(f'Correlation: Median Price vs Price per Square Meter ({latest_year})')
    plt.xlabel('Price per Square Meter (CHF)')
    plt.ylabel('Median Price (CHF)')
    plt.grid(True)
    plt.legend(title='Room Count')
    plt.tight_layout()
    plt.show()

## Conclusions and Insights

Based on our exploratory data analysis, we can draw the following insights:

1. Price trends: [To be completed after running the analysis]
2. Neighborhood impact: [To be completed after running the analysis]
3. Room count effect: [To be completed after running the analysis]
4. Building age correlation: [To be completed after running the analysis]

These insights will guide our machine learning model development to predict real estate prices in Zurich.